## Projektstruktur

Ausgehend von einem beliebigen Wurzelordner liegt dem Projekt folgende Ordnerstruktur zugrunde:

- **master_cloud** [Sciebo-Ordner]
  - ... externer Code, Tutorials etc.
  - *corpora*
    - ... diverse rohe Corpora
    - hpc [backup der ETL und NLP Pipeline Resultate auf dem HPC]
    - *preprocessed* [corpora nach ETL Pipeline]
      - *nlp* [corpora nach NLP Pipeline]
        - spacy_model
          - vocab
    - scrapy
      - faz
      - focus
    + German_datasets.ods [s.u.]
    
- **topiclabeling**
  - data
    - feeds [Rohoutput der scrapy feeds]
    + IWNLP.Lemmatizer_20170501.json
  - logs
  - *src* [Git Projektordner, Python code und Notebooks]
    - scrapy [code]
  + ToDo.txt

    

In [1]:
from os import listdir
from os.path import isfile, join

from constants import DATA_BASE, ETL_PATH, NLP_PATH, HASH
import pandas as pd
import ezodf

pd.set_option('display.precision',10)

## German Corpora Overview

Übersicht über deutschsprachige Corpora, die zur Verfügung stehen. Die ersten 6 fallen hierbei in die engere Wahl. Die gecrawlten News-Corpora von FAZ und Focus können auch als ein gemeinsamer Corpus betrachten werden, ebenso könnten auch weitere Newsseiten hinzugenommen werden. Das Parsing des Wikipedia-Corpus ist noch nicht abgeschlossen. Der dewac-Corpus ist aufgrund seiner Größe noch nicht vollständig durch die NLP-Pipeline gegangen.

In [9]:
def read_ods(filename, sheet_no=0, header=0):
    tab = ezodf.opendoc(filename=filename).sheets[sheet_no]
    return pd.DataFrame({col[header].value:[x.value for x in col[header+1:]]
                         for col in tab.columns()}).iloc[:,:-1].dropna(how='all')

def highlight(s):
    if s.Selected:
        col = '#dd3355' if s.name%2 else '#cc2244'
        return ['color: %s' % col]*len(s)
    return ['']*len(s)

df = read_ods(join(DATA_BASE, 'German_datasets.ods'))
df.style.apply(highlight, axis=1)

,Category,Name,Date,Content,#documents,#sentences,#tokens,Size (unzipped + unprocessed),Remarks,Link,Selected
0,Online-Partizipation,Online-Partizipation,2011-2018,Online-Partizipation,25981,143384,1751790,16M,interessant,https://github.com/Liebeck/OnlineParticipationDatasets,True
1,Politics,German political speeches,1984-2018,includes the four highest ranked functions on federal state level,6037,221397,4136463,73M,interessant,https://hal.archives-ouvertes.fr/hal-01798703,True
2,Politics,Europarl,1996-2011,parallel corpus extracted from the European Parliament web site,12788,2692391,56609511,~3.5G,interessant,"http://opus.nlpl.eu/Europarl.php, http://www.statmt.org/europarl/",True
3,News,faz.de,2006-2018,"finanzen, feuilleton, gesellschaft, technik, politik, wirtschaft, wissen, reise, beruf",51385,1837668,29527792,201M,crawled,http://www.faz.de,True
4,News,focus.de,bis 2018,"politik, finanzen, wissen, gesundheit, kultur, panorama, digital, reisen, auto, immobilien, regional",144127,None,None,190M,crawled,http://www.focus.de,True
5,Wikipedia,dewiki,bis 2018,German Wikipedia,3681494,None,None,19.7G,interessant,https://dumps.wikimedia.org/backup-index.html,True
6,www,dewac_preproc,bis 2018,constructed from the Web limiting the crawl to the .de domain and using medium-frequency words from the SudDeutsche Zeitung corpus and basic German vocabulary lists as seeds,1751871,None,None,10.2G,"interessant, aber wenige Metadaten",http://wacky.sslmit.unibo.it/doku.php?id=corpora#german,True
7,Div.,Wikisource,None,"Sammlung von Texten und Quellen, die entweder urheberrechtsfrei sind oder unter einer freien Lizenz stehen.",nan,None,None,1.9G,"interessant, teilw. sprachlich veraltet, schwierig zu parsen (MediaWiki Markdown; enthält auch Meta Content)",https://de.wikisource.org/wiki/Hauptseite,False
8,Div.,DEREKO / IDS Mannheim,None,"belletristische, wissenschaftliche und populärwissenschaftliche Texte, eine große Zahl von Zeitungstexten sowie eine breite Palette weiterer Textarten",nan,None,42B,None,nicht als Download verfügbar,http://www1.ids-mannheim.de/direktion/kl/projekte/korpora.html,False
9,Div.,DWDS-Kernkorpus,None,ein nach Textsorten und zeitlich über das gesamte Jahrhundert ausgewogenes Korpus,nan,None,None,None,nicht als Download verfügbar,https://www.dwds.de/d/k-referenz/,False


In [2]:
def read(f):
     return pd.read_pickle(f)

## Scrapy spiders

Unter ```../../topiclabeling/src/scrapy``` sind derzeit zwei Spiders für die FAZ und den Focus lauffähig. Der Crawler lässt sich von dort aufrufen durch

```scrapy crawl [-a categories="politik|kultur|panorama|digital"] focus```

Die Categories beziehen sich auf die möglichen Rubriken der jeweiligen News-Seite. Ohne ihre Angabe werden als Default-Kategorien Politik und Wirtschaft gecrawlt.

## ETL Pipeline

Die rohen Corpora wurden durch [ETL_pipeline.ipynb](ETL_pipeline.ipynb) geladen, wenn nötig geparst und in ein einheitliches Datenschema gebracht, wobei unter 'Load' hier das Speichern als pandas DataFrame im pickle-Format zu verstehen ist. Der Output der Pipeline befindet sich im Ordner ```../../master_cloud/corpora/preprocessed```.

In [ ]:
read(join(ETL_PATH, 'dewiki.pickle'))

## NLP Pipeline

Nachdem die Corpora in ein einheitliches Format gebracht wurden durchlaufen sie die Spacy Pipeline für das 'de' language model. s. [NLP_pipeline.ipynb](NLP_pipeline.ipynb)

Die Tokenisierung ist nicht optimal und könnte durch Definition einiger Sonderfälle noch verbessert werden. Dies betrifft insbesondere den Forums-Corpus der Online-Partizipation. Hier macht es sich nachteilig bemerkbar, dass das Sprachmodell auf einem orthographisch korrekten Corpus trainiert wurde.

Die Lemmatisierung wird durch IWNLP durchgeführt. Um einige merkwürdige Ergebnisse abzufangen, habe ich hier einen Wrapper herum gebaut, der bestimmte Sonderfälle abfängt, bzw. bestimmte Wortarten nicht durch IWNLP parsen lässt. Zusätzlich überprüfe ich Suffixe von zusammengesetzten Nomen auf Lemmata. Wo IWNLP keine Ergebnisse findet, wird die Lemmatisierung des Spacy Modells verwendet, ansonsten das ursprüngliche Token. Für IWNLP verwende ich die XML-Datenbank von 2017. Der Versuch, das aktuelle Wiktionary neu zu parsen, schlug nach wenigen Wörtern mit einer OutOfMemory Exception fehl (Build sowohl unter Mono als auch Visual Studio). Die Lemmatisierung ist noch anfällig für Fehler, insbesondere wenn das POS-Tagging nicht korrekt war. Named Entities sind ein Problem, wenn sie von Spacy nicht als solche erkannt wurden.

Der Output der Pipeline befindet sich im Ordner ```../../master_cloud/corpora/preprocessed/nlp```. Das spacy vocabulary wird mit gespeichert.

In [ ]:
read(join(NLP_PATH, 'FAZ_nlp.pickle'))

## Phrase detection

Zu den Token des Corpus versuche ich Phrasen-Token zu generieren. Die Phrase-Detection von Gensim erwies sich dabei als eher problematisch. Die Ergebnisse sind zu willkürlich. Die Named Entities von Spacy liefern zu wenige Ergebnisse. Einigermaßen brauchbar (nach etwas Überarbeitung) sind Spacy's noun chunks. Interessant wäre hier ein Dictionary basierter Ansatz, z.B. auf Basis von Wikipedia-Titeln. Dies könnte aber rechenintensiv werden.

## Topic Modeling

Für das topic modeling werden nur die Tags NOUN, PROPN (Entities) und Phrasen verwendet. Zur Erstellung der Models wird Gensims LDAModel verwendet. Hier muss noch mit den Parametern experimentiert werden. Für das Topic Labeling wäre es wünschenswert möglichst exklusive Topics zu haben, d.h. mit hoher Wahrscheinlichkeitsmasse an den Rändern der Dirichlet-Verteilung, z.B. durch niedrige $\alpha$-Werte. Hier besteht auf jeden Fall noch Experimentierbedarf.

[Gensim_topic_modeling.ipynb](Gensim_topic_modeling.ipynb)

-----

### Appendix: Corpus metrics

In [ ]:
path = NLP_PATH

files = sorted([f for f in listdir(path) if isfile(join(path, f))
                # and f[:4] == 'dewa'
                # and f[:4] == 'dewi'
                # and f[:3] == 'Onl'
                # and f[:3] == 'Eur'
                # and f[:3] == 'FAZ'
                # and f[:3] == 'FOC'
                # and f[:3] == 'Pol'
               ])

dfs = [(name, read(join(path, name))) for name in files]

In [ ]:
overview_list = [(name, df.groupby(HASH).ngroups, df.groupby([HASH, 'sent_idx']).ngroups, df.shape[0]) 
                 for name, df in dfs]
overview = pd.DataFrame(overview_list, columns=['dataset', '#documents', '#sentences', '#tokens'])
overview